In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
temperatures = [
    72, 73, 75, 78, 80, 82, 85, 87, 88, 85,
    81, 78, 76, 74, 73, 72, 71, 72, 74, 76,
    79, 82, 85, 88, 90, 92, 93, 92, 89, 86
]
data = []
labels = []
for t in range(30):
    if t % 4 ==0 and t != 0:
      data.append([x for x in temperatures[t-4:t]])
      labels.append(temperatures[t])
data = torch.tensor(data, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

***The RNN class***

In [4]:
class RNN(nn.Module):
  def __init__(self, i_l, h_l, o_l) -> None:
    super().__init__()
    self.x_to_h = nn.Linear(i_l, h_l)
    self.h_to_h = nn.Linear(h_l, h_l)
    self.tanh = nn.ReLU()
    self.h_to_o = nn.Linear(h_l, o_l)
    # self.h = torch.zeros(h_l, dtype=torch.float32)
  def forward(self, x, h):
    x = self.x_to_h(x)
    a = self.h_to_h(h)
    ht = self.tanh(x + a)
    return ht, self.h_to_o(ht)

In [5]:
model = RNN(4, 8, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
epochs = 1000
for epoch in range(epochs):
    predictions = []  # List to accumulate predictions
    ht = torch.zeros(8, dtype=torch.float32)
    for d in data:
        ht, prediction = model(d, ht)
        predictions.append(prediction)

    # Concatenate all predictions into a single tensor
    predictions = torch.cat(predictions, dim=0)

    loss = criterion(predictions, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 ==0:
      print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')
predictions

Epoch [1/1000], Loss: 2050.0366
Epoch [101/1000], Loss: 17.7927
Epoch [201/1000], Loss: 10.4953
Epoch [301/1000], Loss: 3.2797
Epoch [401/1000], Loss: 1.1013
Epoch [501/1000], Loss: 0.5913
Epoch [601/1000], Loss: 0.4143
Epoch [701/1000], Loss: 0.3458
Epoch [801/1000], Loss: 0.2988
Epoch [901/1000], Loss: 0.2545


tensor([79.9949, 87.9193, 76.2603, 70.3355, 79.6447, 89.3874, 89.4146],
       grad_fn=<CatBackward0>)

In [66]:
labels

tensor([80., 88., 76., 71., 79., 90., 89.])